In [1]:
import pandas as pd
import numpy as np
import csv
import re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import logging
from lmfit import minimize, fit_report, Parameters
from aim2_population_model_spatial_aff_parallel import get_mod_spike
from model_constants import (MC_GROUPS, LifConstants)
from popul_model import pop_model

In [2]:
#Global Variables
lmpars_init_dict = {}
lmpars = Parameters()
lmpars.add('tau1', value=8, vary=False)
lmpars.add('tau2', value=200, vary=False)
lmpars.add('tau3', value=1744.6, vary=False)
lmpars.add('tau4', value=np.inf, vary=False)
lmpars.add('k1', value=.74, vary=False, min=0) #a constant
lmpars.add('k2', value=2.75, vary=False, min=0) #b constant
lmpars.add('k3', value=.07, vary=False, min=0) #c constant
lmpars.add('k4', value=.0312, vary=False, min=0)
lmpars_init_dict['t3f12v3final'] = lmpars


In [58]:
class VF_Population_Model:
    
    def __init__(self, vf_tip_size, aff_type):
        self.vf_tip_size = vf_tip_size
        self.aff_type = aff_type
        self.results = None
        self.stress_data = None
    """
        functino takes in a vf_tip_size (given that that there is data assicated with it) an
        afferent type, and runs the single unit model for all of those coordinates with the data
        
    """
    def spatial_stress_vf_model(self, scaling_factor = 0.3):

        #reading data in 
        coords = pd.read_csv(f"data/vfspatial/{self.vf_tip_size}_spatial_coords.csv", header = None)
        x_coords = coords[0]
        y_coords = coords[1]
        stress_data = pd.read_csv(f"data/vfspatial/3.61_spatial_stress.csv", )
        time = stress_data['Time (ms)'].to_numpy()

        afferent_type = []
        x_pos = []
        y_pos = []
        spikes = []
        mean_firing_frequency = []
        peak_firing_frequency = []
        first_spike_time = []
        last_spike_time = []

        #iterating through each of the coordinates
        for i, row in coords.iloc[1:].iterrows():
            
            #getting stress data
            
            if f"Coord {i} Stress (kPa)" in stress_data.columns:
                stress = stress_data[f"Coord {i} Stress (kPa)"]
            else:
                logging.warning("STRESS VALUE COULD NOT BE INDEXED")

            lmpars = lmpars_init_dict['t3f12v3final']
            if self.aff_type == "RA":
                lmpars['tau1'].value = 2.5
                lmpars['tau2'].value = 200
                lmpars['tau3'].value = 1
                lmpars['k1'].value = 35
                lmpars['k2'].value = 0
                lmpars['k3'].value = 0.0
                lmpars['k4'].value = 0

            groups = MC_GROUPS
            mod_spike_time, mod_fr_inst = get_mod_spike(lmpars, groups, time, stress)

            if len(mod_spike_time) == 0 or len(mod_fr_inst) == 0:
                logging.warning(f"SPIKES COULD NOT BE GENERATED FOR {self.vf_tip_size}")
                continue

            if len(mod_spike_time) != len(mod_fr_inst):
                if len(mod_fr_inst) > 1:
                    mod_fr_inst_interp = np.interp(mod_spike_time, time, mod_fr_inst)
                else:
                    mod_fr_inst_interp = np.zeros_like(mod_spike_time)
            else:
                mod_fr_inst_interp = mod_fr_inst

            features, _ = pop_model(mod_spike_time,mod_fr_inst_interp)

            #appending stuff to lists
            afferent_type.append(self.aff_type)
            x_pos.append(row[0])
            y_pos.append(row[1])
            spikes.append(len(mod_spike_time) if len(mod_spike_time) !=0 else None)
            mean_firing_frequency.append(features["Average Firing Rate"])
            peak_firing_frequency.append(np.max(mod_fr_inst_interp))
            first_spike_time.append(mod_spike_time[0] if len(mod_spike_time) != None else None)
            last_spike_time.append(mod_spike_time[-1])

            
        model_results = {
            'afferent_type': self.aff_type,
            'x_position': x_pos,
            'y_position': y_pos,
            'num_of_spikes' : spikes,
            'mean_firing_frequency' : mean_firing_frequency,
            'peak_firing_frequency' : peak_firing_frequency, 
            'first_spike_time': first_spike_time,
            'last_spike_time' : last_spike_time
        }

        self.results = model_results
        return model_results

    def radial_stress_vf_model(self,scaling_factor = 0.3):
        """ Read in the Radial which has sample stress traces for every 2mm from a center point
        to calculate firing"""

        #regex pattern for exstracting the distance from the middle point
        distance_regex = r'\d\.\d{2}'

        #reading data from spatial stress data file for 50 (or n) data points in a grid
        coords = pd.read_csv(f"data/vfspatial/{self.vf_tip_size}_spatial_coords.csv", header = None)
        stress_df = pd.read_csv(f"data/vfspatial/{self.vf_tip_size}_spatial_stress.csv", )
        time = stress_df['Time (ms)'].to_numpy()

        #Reading in the radial stress file
        radial_stress = pd.read_csv(f"data/vfspatial/{self.vf_tip_size}_radial_stress.csv")
        radial_time = stress_df['Time (ms)'].to_numpy()
        
        stress_data = {}
        iff_data = {}



        #Outer loop top iterate through all n spatial points
        for i, row in coords.iloc[1:].iterrows():
            #Flag to ensure that only 0.00mm or only the first stress trace in the radial file gets a scaling factor calculated
            radial_spatial_flag = True
            stress_data[i] = {}
            iff_data[i] = {}
            afferent_type = []
            x_pos = []
            y_pos = []
            spikes = []
            mean_firing_frequency = []
            peak_firing_frequency = []
            first_spike_time = []
            last_spike_time = []

            distance_scaling_factor = 0 

            if f"Coord {i} Stress (kPa)" in stress_df.columns:

                spatial_stress = stress_df[f"Coord {i} Stress (kPa)"]
                spatial_stress_max = np.max(spatial_stress)

                #inner loop to iterate through all radial distances from the center
                for col in radial_stress.columns[1:]:
                    #stores the distacnef rom the cetner in mm
                    distance_from_center = float(re.findall(distance_regex, col)[0])

                    
                    #scaling factor only calculated for the first stress trace in the radial file
                    if radial_spatial_flag:
                        #getting stress data
                        radial_stress_vals = radial_stress[col]
                        radial_stress_max = np.max(radial_stress_vals)
                        distance_scaling_factor = spatial_stress_max / radial_stress_max
                        radial_spatial_flag = False
                    
                    #Performing Scaling 
                    scaled_stress = radial_stress[col] * distance_scaling_factor * scaling_factor

                    stress_data[i][distance_from_center] = {
                        "Time": radial_time,
                        distance_from_center: scaled_stress.to_numpy()
                    }



                    #Calculating spikes and IFFs
                    lmpars = lmpars_init_dict['t3f12v3final']
                    if self.aff_type == "RA":
                        lmpars['tau1'].value = 2.5
                        lmpars['tau2'].value = 200
                        lmpars['tau3'].value = 1
                        lmpars['k1'].value = 35
                        lmpars['k2'].value = 0
                        lmpars['k3'].value = 0.0
                        lmpars['k4'].value = 0

                    groups = MC_GROUPS
                    mod_spike_time, mod_fr_inst = get_mod_spike(lmpars, groups, stress_data[i][distance_from_center]["Time"], stress_data[i][distance_from_center][distance_from_center])

                    if len(mod_spike_time) == 0 or len(mod_fr_inst) == 0:
                        logging.warning(f"SPIKES COULD NOT BE GENERATED FOR {self.vf_tip_size}")
                        continue

                    if len(mod_spike_time) != len(mod_fr_inst):
                        if len(mod_fr_inst) > 1:
                            mod_fr_inst_interp = np.interp(mod_spike_time, time, mod_fr_inst)
                        else:
                            mod_fr_inst_interp = np.zeros_like(mod_spike_time)
                    else:
                        mod_fr_inst_interp = mod_fr_inst

                    features, _ = pop_model(mod_spike_time,mod_fr_inst_interp)

                    #appending stuff to lists
                    afferent_type.append(self.aff_type)
                    x_pos.append(row[0])
                    y_pos.append(row[1])
                    spikes.append(len(mod_spike_time) if len(mod_spike_time) !=0 else None)
                    mean_firing_frequency.append(features["Average Firing Rate"])
                    peak_firing_frequency.append(np.max(mod_fr_inst_interp))
                    first_spike_time.append(mod_spike_time[0] if len(mod_spike_time) != None else None)
                    last_spike_time.append(mod_spike_time[-1])

                    iff_data[i][distance_from_center] = {
                        'afferent_type': self.aff_type,
                        'x_position': x_pos,
                        'y_position': y_pos,
                        'num_of_spikes' : spikes,
                        'mean_firing_frequency' : mean_firing_frequency,
                        'peak_firing_frequency' : peak_firing_frequency, 
                        'first_spike_time': first_spike_time,
                        'last_spike_time' : last_spike_time
                    }
            else:
                logging.warning("STRESS VALUE COULD NOT BE INDEXED")
        self.stress_data = stress_data
        self.results = iff_data

        

    def aggregate_results(self):
        df = pd.DataFrame(self.results)
        file_path = f"generated_csv_files/{self.vf_tip_size}_vf_popul_model.csv"
        df.to_csv(file_path, index = False)
        return file_path 
    
    def plot(self):
        """
        Plots the iffs on a grid for the original n points, the magniude of the peak firing
        frequency directly affects the size of the circle plotted, and the opacity
        """
        #colors for differnet afferents
        colors = {'SA': '#31a354', 'RA': '#3182bd'}
        plt.figure(figsize=(10, 5))

        # Plot the stimulus locations as circles
        x_positions = self.results.get("x_position")
        y_positions = self.results.get("y_position")
        mean_iffs = self.results.get("mean_firing_frequency")
        peak_iffs = self.results.get("peak_firing_frequency")
        
        x_positions = [float(value) for value in x_positions]
        y_positions = [float(value) for value in y_positions]
        #scaling peak_iffs so it looks better when plotting
    
        alphas = [float(value)/max(peak_iffs) for value in peak_iffs]

        #Scatter plot
                # Plot the stimulus locations as circles
        for x_pos, y_pos, radius, alpha in zip(x_positions, y_positions, peak_iffs, alphas):
            plt.gca().add_patch(
                patches.Circle((x_pos, y_pos), radius*2, edgecolor='black', facecolor = colors.get(self.aff_type) , linewidth=1, alpha = 0.5)
            )
        plt.xlabel('Length (mm)')
        plt.ylabel('Width (mm)')
        plt.title("VF Afferent Stress Distribution")
        plt.gca().set_aspect('equal', adjustable='box')
        plt.xlim(min(x_positions) - 1, max(x_positions) + 1)
        plt.ylim(min(y_positions) - 1, max(y_positions) + 1)
        plt.savefig(f"vf_graphs/aggregated_results_on_grid/{self.vf_tip_size}_{self.aff_type}_constant_opacity.png")

    def get_iffs(self):
        return self.results
    
    def get_stress_traces(self):
        return self.stress_data


In [42]:
#creates model class
vf_model = VF_Population_Model(4.17, "SA")

#runs the model which calculates the results
# vf_model.spatial_stress_vf_model()
vf_model.radial_stress_vf_model()
iffs_results = vf_model.get_iffs()
stress_results = vf_model.get_stress_traces()


# #writing the data to csv
# vf_model.aggregate_results()

# #plotting the aggregated_results
# vf_model.plot()


MAX SCALED STRESS 84.797187
MEAN SCALED STRESS 69.28609257967786
MAX SCALED STRESS 53.77216799999999
MEAN SCALED STRESS 43.936168907033036
MAX SCALED STRESS 43.411254
MEAN SCALED STRESS 35.47047056587238
MAX SCALED STRESS 37.249344
MEAN SCALED STRESS 30.435696705905652
MAX SCALED STRESS 30.080990999999997
MEAN SCALED STRESS 24.5785800210557
MAX SCALED STRESS 24.0818019
MEAN SCALED STRESS 19.676760870497702
MAX SCALED STRESS 18.4435335
MEAN SCALED STRESS 15.069844003048706
MAX SCALED STRESS 14.719182
MEAN SCALED STRESS 12.026750705582996
MAX SCALED STRESS 11.550413099999998
MEAN SCALED STRESS 9.437612577335944
MAX SCALED STRESS 8.5017126
MEAN SCALED STRESS 6.946580208637918
MAX SCALED STRESS 5.772590999999999
MEAN SCALED STRESS 4.716669257952389
MAX SCALED STRESS 4.2675879
MEAN SCALED STRESS 3.486961104245319
MAX SCALED STRESS 3.2812913999999997
MEAN SCALED STRESS 2.6810779888142555
MAX SCALED STRESS 2.7725279099999995


MEAN SCALED STRESS 2.2653773547928293
MAX SCALED STRESS 3.1950068999999996
MEAN SCALED STRESS 2.610576493856729
MAX SCALED STRESS 2.38658382
MEAN SCALED STRESS 1.9500301181517894
MAX SCALED STRESS 2.79395901
MEAN SCALED STRESS 2.282888288128717
MAX SCALED STRESS 3.8459409
MEAN SCALED STRESS 3.1424417168210885
MAX SCALED STRESS 4.3475538
MEAN SCALED STRESS 3.5522997390380207
MAX SCALED STRESS 3.1891107
MEAN SCALED STRESS 2.605758823364648
MAX SCALED STRESS 3.1507520999999996
MEAN SCALED STRESS 2.5744169030359014
MAX SCALED STRESS 3.2635602
MEAN SCALED STRESS 2.666590125342275
MAX SCALED STRESS 3.4709844
MEAN SCALED STRESS 2.836072294143654
MAX SCALED STRESS 3.538935
MEAN SCALED STRESS 2.891593226610141
MAX SCALED STRESS 2.80346226
MEAN SCALED STRESS 2.290653198769764
MAX SCALED STRESS 16.3320117
MEAN SCALED STRESS 13.344561472995348
MAX SCALED STRESS 10.356566154846215
MEAN SCALED STRESS 8.462144205831258
MAX SCALED STRESS 8.361045139854365
MEAN SCALED STRESS 6.8316433690935225
MAX SCAL

MEAN SCALED STRESS 1.8176923608694446
MAX SCALED STRESS 1.6374372141995395
MEAN SCALED STRESS 1.3379173679719225
MAX SCALED STRESS 1.1118060290291787
MEAN SCALED STRESS 0.9084345864669867
MAX SCALED STRESS 0.8219411277590898
MEAN SCALED STRESS 0.671591730418834


MAX SCALED STRESS 0.6319795671513181
MEAN SCALED STRESS 0.5163779440222102
MAX SCALED STRESS 0.5339912780915308
MEAN SCALED STRESS 0.43631364167058434
MAX SCALED STRESS 0.6153610976786379
MEAN SCALED STRESS 0.5027992949979943
MAX SCALED STRESS 0.45965811190494676
MEAN SCALED STRESS 0.37557749061896833


MAX SCALED STRESS 0.5381189265233576
MEAN SCALED STRESS 0.4396862626057534
MAX SCALED STRESS 0.740731550166971
MEAN SCALED STRESS 0.6052369978462859
MAX SCALED STRESS 0.8373426293961788
MEAN SCALED STRESS 0.684176008101258
MAX SCALED STRESS 0.6142254844490913
MEAN SCALED STRESS 0.5018714074863081


...

MAX SCALED STRESS 0.4142327785063403
MEAN SCALED STRESS 0.3384613602288995
MAX SCALED STRESS 0.3250560874448751
MEAN SCALED STRESS 0.2655968572422198
MAX SCALED STRESS 0.2392584066397414
MEAN SCALED STRESS 0.19549328359016402
MAX SCALED STRESS 0.16245443592658163
MEAN SCALED STRESS 0.13273828749566763


MAX SCALED STRESS 0.1201000703257177
MEAN SCALED STRESS 0.09813137623782864
MAX SCALED STRESS 0.09234334175030647
MEAN SCALED STRESS 0.07545190926362076
MAX SCALED STRESS 0.07802552748146442
MEAN SCALED STRESS 0.063753105032683
MAX SCALED STRESS 0.08991509076618041
MEAN SCALED STRESS 0.07346782956781246


MAX SCALED STRESS 0.06716414314986224
MEAN SCALED STRESS 0.05487848400903412
MAX SCALED STRESS 0.07862864959106586
MEAN SCALED STRESS 0.06424590433158184
MAX SCALED STRESS 0.10823392121778067
MEAN SCALED STRESS 0.0884357815300493
MAX SCALED STRESS 0.12235050088244022
MEAN SCALED STRESS 0.09997016077313718


MAX SCALED STRESS 0.0897491576790952
MEAN SCALED STRESS 0.07333224886544308
MAX SCALED STRESS 0.08866965547186567
MEAN SCALED STRESS 0.07245021270735352
MAX SCALED STRESS 0.09184434362376305
MEAN SCALED STRESS 0.0750441863384868
MAX SCALED STRESS 0.0976817537933944
MEAN SCALED STRESS 0.0798138175374132


MAX SCALED STRESS 0.09959404523996887
MEAN SCALED STRESS 0.08137630858622664
MAX SCALED STRESS 0.0788960936414445
MEAN SCALED STRESS 0.0644644273792413


In [57]:
iffs_results.get(1).get(0.0).get('x_position')

['8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '8.8687',
 '3.27558',
 '3.27558',
 '3.27558',
 '3.27558',
 '3.27558',
 '3.27558',
 '3.27558',
 '3.27558',
 '5.22687',
 '5.22687',
 '5.22687',
 '5.22687',
 '5.22687',
 '5.22687',
 '5.22687',
 '5.22687',
 '7.48174',
 '7.48174',
 '7.48174',
 '7.48174',
 '7.48174',
 '7.48174',
 '9.01573',
 '9.01573',
 '9.01573',
 '9.01573',
 '9.01573',
 '10.9695',
 '10.9695',
 '10.9695',
 '10.9695',
 '12.8036',
 '12.8036',
 '12.8036',
 '12.8036',
 '14.8462',
 '14.8462',
 '3.01684',
 '3.01684',
 '3.01684',
 '3.01684',
 '5.00062',
 '5.00062',
 '5.00062',
 '5.00062',
 '5.00062',
 '5.00062',
 '5.00062',
 '5.00062',
 '5.00062',
 '7.15666',
 '7.15666',
 '7.15666',
 '7.15666',
 '7.15666',
 '7.15666',
 '9.11605',
 '9.11605',
 '9.11605',
 '9.11605',
 '9.11605',
 '9.